In [ ]:
import plotly.graph_objects as go
import numpy as np


In [ ]:
def hcp(n):
    num_points = n*n*n
    points = np.zeros((num_points, 3))
    for i in range(num_points):
        j = (i % (n*n)) // n
        k = i // (n*n)
        l = i % n
        points[i] = np.array([
            np.sqrt(3) * (j + (k % 2)/3), 
            2 * np.sqrt(6) / 3 * k, 
            2 * l + (j + k) % 2])
    return points

def hcp_ipos(n):
    num_points = n*n*n
    ipos_list = np.zeros((num_points, 3), np.int32)
    for i in range(num_points):
        j = (i % (n*n)) // n
        k = i // (n*n)
        l = i % n
        ipos_list[i] = np.array([j, k, l], np.int32)
    return ipos_list

kHcpStep0 = 1.73205080756887729352744634150587236694280525381038062805581
kHcpStepInv0 = 0.577350269189625764509148780501957455647601751270126876018602
kHcpStep1 = 1.63299316185545206546485604980392759464396498710444675228846
kHcpStepInv1 = 0.612372435695794524549321018676472847991486870164167532108173

    
def get_hcp_cell_center(ipos):
    return np.array([
        kHcpStep0 * (ipos[0] + (ipos[1] % 2) / 3),
        kHcpStep1 * ipos[1],
        (ipos[2] * 2 + (ipos[0] + ipos[1]) % 2)
    ])

coord = hcp(6)
ipos_list = hcp_ipos(6)

fig = go.Figure(data=go.Scatter3d(
    x=coord[:,0], y=coord[:,1], z=coord[:,2], mode='markers',
    marker=dict(size=coord[:,0]*0 + 30, symbol="circle", color=-coord[:,2], opacity=1),
))
fig.show()

In [ ]:
def approximate_hcp_ipos(x):
    k = int(np.round(x[1]*3/(2*np.sqrt(6))))
    j = int(np.round(x[0]/(np.sqrt(3))-(k%2)/3.0))
    l = int(np.round((x[2] - ((j+k)%2))/2))
    return np.array([j, k, l], np.int32)

In [ ]:
for i, x in enumerate(coord):
    print(approximate_hcp_ipos(x), ipos_list[i])
    print(np.sum(approximate_hcp_ipos(x)-ipos_list[i]))

In [ ]:
for i, ipos in enumerate(ipos_list):
    print(get_hcp_cell_center(ipos), coord[i])

In [ ]:
test_points = np.random.uniform(4,6, (1000000,3))

In [ ]:
closest_ipos = np.zeros((len(test_points), 3), np.int32)
for i, test_point in enumerate(test_points):
    displacements = np.tile(test_point, (len(coord), 1)) - coord
    distance_sqr = np.sum(displacements*displacements, axis=1)
    closest_id = np.argmin(distance_sqr)
    closest_ipos[i] = ipos_list[closest_id]

In [ ]:
approximate_closest_ipos = np.zeros((len(test_points), 3), np.int32)
for i, test_point in enumerate(test_points):
    approximate_ipos = approximate_hcp_ipos(test_point)
    approximate_closest_ipos[i] = approximate_ipos

In [ ]:
approximate_discrepancy = closest_ipos - approximate_closest_ipos

In [ ]:
possible_offsets = []
occupied_encoded_offset = np.zeros(1000, np.int32)
def encode_offset(diff_ipos):
    return (diff_ipos[0] + 5)*100 + (diff_ipos[1] + 5)*10 + (diff_ipos[2]+5)
for diff_ipos in approximate_discrepancy:
    encoded = encode_offset(diff_ipos)
#     print(diff_ipos, encoded)
    if occupied_encoded_offset[encoded] == 0:
        possible_offsets.append(diff_ipos)
    occupied_encoded_offset[encoded]+=1

In [63]:
possible_offsets, len(possible_offsets)

([array([0, 0, 0], dtype=int32),
  array([1, 0, 1], dtype=int32),
  array([ 0, -1,  1], dtype=int32),
  array([0, 1, 0], dtype=int32),
  array([ 0,  1, -1], dtype=int32),
  array([-1,  1,  0], dtype=int32),
  array([ 0, -1,  0], dtype=int32),
  array([-1,  0, -1], dtype=int32),
  array([1, 1, 0], dtype=int32),
  array([-1, -1,  0], dtype=int32),
  array([ 0, -1, -1], dtype=int32),
  array([1, 0, 0], dtype=int32),
  array([-1,  0,  0], dtype=int32),
  array([ 1, -1,  0], dtype=int32),
  array([0, 1, 1], dtype=int32),
  array([-1,  0,  1], dtype=int32),
  array([ 1,  0, -1], dtype=int32)],
 17)

In [70]:
frequencies = np.zeros(len(possible_offsets), np.int32)
for i, diff_ipos in enumerate(possible_offsets):
    frequencies[i] = occupied_encoded_offset[encode_offset(diff_ipos)]
    print(diff_ipos, frequencies[i])
    

[0 0 0] 828390
[1 0 1] 14075
[ 0 -1  1] 11342
[0 1 0] 19683
[ 0  1 -1] 14107
[-1  1  0] 3958
[ 0 -1  0] 28865
[-1  0 -1] 9973
[1 1 0] 5828
[-1 -1  0] 5624
[ 0 -1 -1] 17475
[1 0 0] 14196
[-1  0  0] 12144
[ 1 -1  0] 5667
[0 1 1] 5837
[-1  0  1] 2584
[ 1  0 -1] 252


In [67]:
## inspect rare case
# target_encoded = encode_offset([ 1,  0, -1])
# for i, test_point in enumerate(test_points):
#     diff_pos = approximate_discrepancy[i]
#     if encode_offset(diff_pos) ==target_encoded :
#         closest_center = get_hcp_cell_center(closest_ipos[i])
#         diff_closest = closest_center - test_point
#         distance_closest_sqr = np.sum(diff_closest * diff_closest)
#         approx_center = get_hcp_cell_center(approximate_closest_ipos[i])
#         diff_approx = approx_center - test_point
#         distance_approx_sqr = np.sum(diff_approx * diff_approx)
#         print(distance_closest_sqr, distance_approx_sqr)

In [71]:
frequencies

array([828390,  14075,  11342,  19683,  14107,   3958,  28865,   9973,
         5828,   5624,  17475,  14196,  12144,   5667,   5837,   2584,
          252], dtype=int32)

In [72]:
np.argsort(frequencies)

array([   252,   2584,   3958,   5624,   5667,   5828,   5837,   9973,
        11342,  12144,  14075,  14107,  14196,  17475,  19683,  28865,
       828390], dtype=int32)

In [80]:
sorted_idx = np.argsort(frequencies)
sorted_possible_offsets = np.zeros_like(possible_offsets)
for i, sorted_id in enumerate(reversed(sorted_idx)):
    print(frequencies[sorted_id], possible_offsets[sorted_id])
    sorted_possible_offsets[i] = possible_offsets[sorted_id]

828390 [0 0 0]
28865 [ 0 -1  0]
19683 [0 1 0]
17475 [ 0 -1 -1]
14196 [1 0 0]
14107 [ 0  1 -1]
14075 [1 0 1]
12144 [-1  0  0]
11342 [ 0 -1  1]
9973 [-1  0 -1]
5837 [0 1 1]
5828 [1 1 0]
5667 [ 1 -1  0]
5624 [-1 -1  0]
3958 [-1  1  0]
2584 [-1  0  1]
252 [ 1  0 -1]


In [81]:
sorted_possible_offsets

array([[ 0,  0,  0],
       [ 0, -1,  0],
       [ 0,  1,  0],
       [ 0, -1, -1],
       [ 1,  0,  0],
       [ 0,  1, -1],
       [ 1,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  1],
       [-1,  0, -1],
       [ 0,  1,  1],
       [ 1,  1,  0],
       [ 1, -1,  0],
       [-1, -1,  0],
       [-1,  1,  0],
       [-1,  0,  1],
       [ 1,  0, -1]], dtype=int32)

In [83]:
faster_closest_ipos = np.zeros((len(test_points), 3), np.int32)
for i, test_point in enumerate(test_points):
    approximate_ipos = approximate_hcp_ipos(test_point)
    faster_closest_ipos[i] = approximate_ipos
    
    min_distance_sqr = 999.9
    min_jpos = None
    for ipos_offset in sorted_possible_offsets:
        jpos = approximate_ipos + ipos_offset
        jcenter = get_hcp_cell_center(jpos)
        diff = jcenter - test_point
        distance_sqr = np.sum(diff*diff)
        if distance_sqr<min_distance_sqr:
            min_distance_sqr = distance_sqr
            min_jpos = jpos
    faster_closest_ipos[i] = min_jpos

In [84]:
np.sum(closest_ipos - faster_closest_ipos)

0

In [12]:
eps =1e-3

coord = hcp(12)
ipos = hcp_ipos(12)

ref = np.array([np.sqrt(48), np.sqrt(128/3), 8])
ref_ipos = np.array([4, 4, 4], np.int32)
for i in range(len(coord)):
    x = coord[i]
    relative_ipos = ipos[i]-ref_ipos
    displacement = x - ref
    dist_sqr = np.dot(displacement,displacement)
    assigned_mat = None
    if dist_sqr < 20-eps:
        print('jpos = ipos + I3{', f'{relative_ipos[0]}, {relative_ipos[1]}, {relative_ipos[2]}', '};')
        print('jpos_linear = linearize_hcp_ipos(jpos);')
        print('if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;')

jpos = ipos + I3{ -1, -2, -1 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ -1, -2, 0 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ 0, -2, -1 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ 0, -2, 0 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ 0, -2, 1 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ 1, -2, -1 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ 1, -2, 0 };
jpos_linear = linearize_hcp_ipos(jpos);
if (within_hcp_grid(jpos)) need_ghost(jpos_linear) = jpos_linear;
jpos = ipos + I3{ -2, -1, -1 };
jpos_linear